## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-31-20-45-00 +0000,wsj,Opinion | Doping Casts a Shadow on the New Yor...,https://www.wsj.com/opinion/doping-casts-a-sha...
1,2025-10-31-20-41-58 +0000,nyt,"In One Jamaican Town, Hurricane Melissa Left B...",https://www.nytimes.com/2025/10/31/world/ameri...
2,2025-10-31-20-40-00 +0000,wsj,"Opinion | At the National Symphony Orchestra, ...",https://www.wsj.com/opinion/at-the-national-sy...
3,2025-10-31-20-39-00 +0000,wsj,Opinion | Bill Gates Apologizes for Earth’s Su...,https://www.wsj.com/opinion/bill-gates-apologi...
4,2025-10-31-20-37-04 +0000,nypost,Armed Alabama man charged with threatening rab...,https://nypost.com/2025/10/31/us-news/alabama-...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2330/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
98,trump,55
3,new,20
339,you,11
267,andrew,11
97,shutdown,11


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
290,2025-10-30-23-41-00 +0000,wsj,A day after President Trump vowed to resume te...,https://www.wsj.com/politics/national-security...,121
268,2025-10-31-00-58-02 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...,120
144,2025-10-31-14-56-14 +0000,nypost,Trump’s call to go ‘nuclear’ to end government...,https://nypost.com/2025/10/31/us-news/trumps-c...,114
110,2025-10-31-16-22-00 +0000,wsj,President Trump urged Senate Republicans to en...,https://www.wsj.com/politics/policy/filibuster...,112
244,2025-10-31-04-57-01 +0000,nypost,Trump demands Senate Republicans invoke ‘nucle...,https://nypost.com/2025/10/31/us-news/trump-de...,106


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
290,121,2025-10-30-23-41-00 +0000,wsj,A day after President Trump vowed to resume te...,https://www.wsj.com/politics/national-security...
10,78,2025-10-31-20-26-00 +0000,wsj,The federal government pays out $100 billion a...,https://www.wsj.com/politics/policy/what-to-kn...
268,58,2025-10-31-00-58-02 +0000,nyt,"Trump and Xi Ease Off the Trade War, but New N...",https://www.nytimes.com/2025/10/30/us/politics...
180,57,2025-10-31-11-41-06 +0000,nypost,Prince Andrew thrown ‘under the bus’ by royal ...,https://nypost.com/2025/10/31/world-news/princ...
289,49,2025-10-30-23-42-12 +0000,nypost,Amazon shares soar as AI demand boosts cloud r...,https://nypost.com/2025/10/30/business/amazon-...
110,44,2025-10-31-16-22-00 +0000,wsj,President Trump urged Senate Republicans to en...,https://www.wsj.com/politics/policy/filibuster...
123,41,2025-10-31-16-00-00 +0000,wsj,Hate Giant Airports? You Can Get Help Navigati...,https://www.wsj.com/lifestyle/travel/airport-v...
68,39,2025-10-31-18-12-27 +0000,bbc,'No inch here was spared': Hurricane Melissa d...,https://www.bbc.com/news/videos/ce3xxzg09gro?a...
107,35,2025-10-31-16-31-57 +0000,nypost,Ex-CBS producer alleges ‘race-based layoffs’ a...,https://nypost.com/2025/10/31/media/ex-cbs-pro...
45,34,2025-10-31-19-05-54 +0000,nyt,Trump Administration Must Pay SNAP Benefits Du...,https://www.nytimes.com/2025/10/31/us/politics...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
